In [12]:
CARDIAC_COMA_REPO = "/home/rodrigo/CISTIB/repos/CardiacCOMA/"

In [13]:
import mlflow
import os, sys

import torch
import torch.nn.functional as F

import os; os.chdir(CARDIAC_COMA_REPO)
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
from mlflow.tracking import MlflowClient

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

#import surgeon_pytorch
#from surgeon_pytorch import Inspect, get_layers

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

import model.Model3D
from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy
from pprint import pprint

from copy import deepcopy
from typing import List

In [14]:
from mlflow_helpers import \
    list_artifacts,\
    get_significant_loci,\
    get_metrics_cols, \
    get_params_cols, \
    get_runs_df, \
    get_good_runs,\
    summarize_loci_across_runs,\
    get_model_pretrained_weights

In [15]:
TRACKING_URI = f"file://{CARDIAC_COMA_REPO}/mlruns"
mlflow.set_tracking_uri(TRACKING_URI)

In [16]:
client = MlflowClient()

### Retrieve cardiac indices

In [17]:
timeframe = "1".zfill(3)
datafolder = "data/cardio/cardiac_indices"

In [18]:
df = pd.concat([pd.read_csv(f"{datafolder}/G{i}/LVRV_time{timeframe}.csv", index_col="case_id") for i in range(1,5)])

In [19]:
df

,LVEDV (mL),LVM (g),RVEDV (mL)
case_id,,,
1575594,169.515362,96.505318,203.963810
1982723,113.870137,62.877573,121.807408
1070966,131.174275,72.171091,137.144004
1851234,156.977255,92.898278,178.114947
1752613,119.700546,94.632015,141.645332
...,...,...,...
4542967,151.443686,89.504582,154.916932
4199605,147.178375,107.944423,144.030801
4934524,170.701413,102.833356,179.698969


### Compute sphericity

In [1]:
import scipy
import math
import pandas as pd
import os
import shlex
from subprocess import check_output

from scipy.spatial import ConvexHull

import numpy as np
import pickle as pkl

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

# meshes = pkl.load(open("data/cardio/LV_meshes_at_ED_35k.pkl", "rb"))



In [34]:
convhull = {}
for i, (id, mesh) in enumerate(meshes.items()):
    if i % 1000 == 0:
        print(i)
    convhull[id] = ConvexHull(mesh)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


In [ ]:
def sphericity_index(convex_hull):
    return((36 * math.pi * convex_hull.volume**2)**(1./3) / convex_hull.area)

In [8]:
sph_df = pd.read_csv("data/cardio/sphericity.csv")

In [26]:
df.index = df.index.astype(str)
sph_df = sph_df.set_index("id")
sph_df.index = sph_df.index.astype(str)

In [37]:
cardiac_indices_df = df.merge(sph_df, left_index=True, right_index=True)

In [38]:
cardiac_indices_df

,LVEDV (mL),LVM (g),RVEDV (mL),LVSph
1000215,109.544212,70.061720,118.586674,0.945627
1000336,123.185025,73.821846,121.557099,0.935885
1000363,172.502476,136.292264,188.630958,0.939892
1000380,179.841796,107.327191,200.192059,0.943578
1000407,140.523982,85.071837,150.744855,0.948078
...,...,...,...,...
4999331,119.871666,77.005132,128.292674,0.951050
4999373,143.356482,83.120939,137.664470,0.946512
4999543,123.396442,84.933799,139.303552,0.955392
4999719,137.733007,84.338143,138.312118,0.950610


In [39]:
def experiment_selection_widget():
    options = [exp.name for exp in mlflow.list_experiments()]

    experiment_w = widgets.Select(
      options=options,
      value=options[1]
    )
    
    return experiment_w

exp_w = experiment_selection_widget()

@interact
def get_runs(exp_name=exp_w):  
  try:
    exp_id = mlflow.get_experiment_by_name(exp_name).experiment_id
    runs_df = get_runs_df(exp_name=exp_name, only_finished=True)
    metrics, params = get_metrics_cols(runs_df), get_params_cols(runs_df)  
    # display(runs_df.loc[:, [*metrics, *params]].drop("params.platform", axis=1).head(10))
    return runs_df
  except:
    pass

A Jupyter Widget

In [42]:
runs_df = get_runs_df(exp_name=exp_w.value, only_finished=True)

In [86]:
z_paths = runs_df.reset_index().apply(
    lambda row: ((row.experiment_id, row.run_id), row.artifact_uri.replace("file://", "") + f'''/output/z_adj_{row.experiment_id}_{row.run_id}.tsv'''), axis=1
)
z_paths = dict(z_paths.values.tolist())

In [97]:
z_dfs = {}

for run_id, z_path in z_paths.items():
    try:
        z_dfs[run_id] = pd.read_csv(z_path, sep="\t").set_index("ID")
    except FileNotFoundError:
        pass

In [ ]:
for run in z_dfs:
    

In [98]:
list(z_dfs.values())

[             z000      z001      z002      z003      z004      z005      z006  \
 ID                                                                              
 2471198  0.754081  0.217631  0.892939  0.801908  0.398927 -0.485893  0.287335   
 1746709 -0.985621 -0.575624 -2.010317 -1.195979 -1.948443  2.083467  1.517243   
 2008596 -1.323050 -2.476096 -1.314485 -1.831838 -2.217431  1.894083  0.395576   
 4917788  0.275106  0.853365  0.618360  1.108492  0.711543 -0.204171  0.523633   
 1780889 -1.405695  0.349099 -0.181581 -0.567419 -0.055743  0.754549  0.506033   
 ...           ...       ...       ...       ...       ...       ...       ...   
 4485380 -0.514773  0.138087 -0.526865 -1.267517 -0.258839  0.301744 -1.159779   
 3878790  1.341516  0.296811  0.339539  0.015170  0.486765 -0.661380  0.471133   
 1703522  0.263136 -0.374577  0.801229  0.168929  0.741050 -0.950089  0.142424   
 2034360 -0.148399  0.756516  0.677940 -0.222246  1.546598 -1.512587 -1.562863   
 4424098  0.1929